In [ ]:
# Change directory to the root so that relative path loads work correctly
import os

try:
    os.chdir(os.path.join(os.getcwd(), ".."))
    print(os.getcwd())
except:
    pass

In [ ]:
from datetime import datetime

import torch
from torch import nn

from experiments.A_proof_of_constraint.experiment_definition import dictionary_product
from experiments.A_proof_of_constraint.main import run_experiment
from experiments.A_proof_of_constraint.reductions import Huber_Reduction, Lp_Reduction

In [ ]:
# Saving utilities
def get_savefile():
    base_name = "proof-of-constraint"
    time_string = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    savefile = f"{base_name}_{time_string}.pth"
    return savefile


def save_out(
    summary, savefile, directory="/global/u1/g/gelijerg/Projects/pyinsulate/results"
):
    full_file = f"{directory}/{savefile}"
    print(f"Saving to file {full_file}")
    torch.save(summary, full_file)

In [ ]:
base_configuration = {
    "training_parameterizations": {
        "amplitudes": [1.0],
        "frequencies": [1.0],
        "phases": [0.0],
        "num_points": 500,
        "sampling": "uniform",
    },
    "testing_parameterizations": {
        "amplitudes": [1.0],
        "frequencies": [1.0],
        "phases": [0.0],
        "num_points": 500,
        "sampling": "uniform",
    },
    "batch_size": 100,
    "model_size": [20, 20, 20],
    "method": "constrained",
    "learning_rate": 1e-3,
    "ground_approximation": None,
    "reduction": Huber_Reduction(6),
    "model_act": nn.Tanh(),
}

configuration_revisions = list(
    dictionary_product(
        **{
            #             "method": ["unconstrained", "soft-constrained", "reduction", "approximate"],
            "method": ["unconstrained"]
        }
    )
)
num_epochs = 400
save_interval = 10

In [ ]:
# Run experiment
all_savefiles = list()
final_checkpoints = list()
for revision in configuration_revisions:
    configuration = base_configuration.copy()
    configuration.update(revision)

    savefile = get_savefile()
    all_savefiles.append(savefile)
    print(f"Running proof of constraint with savefile {savefile}")
    checkpoint_save_file_base = os.path.splitext(savefile)[0]
    final_checkpoints.append(f"{checkpoint_save_file_base}_{num_epochs:05d}.pth")
    final_result = run_experiment(
        num_epochs,
        log=print,
        save_directory="results/checkpoints",
        save_file=checkpoint_save_file_base,
        save_interval=save_interval,
        **configuration,
    )
    print(f"Completed run with savefile {savefile}")
    # Save out
    configuration, (trainer, train_evaluator, test_evaluator), (
        training_monitor,
        evaluation_train_monitor,
        evaluation_test_monitor,
    ) = final_result

    save_out(
        {
            "configuration": configuration,
            "training_monitor": training_monitor,
            "evaluation_train_monitor": evaluation_train_monitor,
            "evaluation_test_monitor": evaluation_test_monitor,
        },
        savefile=savefile,
    )
print("")
print(f"Files were saved to {all_savefiles}")
print(f"Checkpoints were saved to {final_checkpoints}")
print(f"Corresponding revisions {configuration_revisions}")

In [ ]:
print("done!")